In [1]:
###Start importing the functions for preprocessing
###################################################

In [2]:
#try importing to add extra data
import pandas as pd

def preprocess_process_csv(filename):
    #
    df = pd.read_csv(filename,header=None)
    
    #create the correct headers
    df.columns = ["Action", "ParentProcessGuid", "Hostname", "unknown", "Object", "ProcessGuid", "Pid", "Ppid", "Principal", "Acuity_level", "CommandLine", "Image_path", "Parent_image_path", "Sid", "User", "Tid", "Timestamp"]
    
    #create the correct headers
    df['CommandLine_length']  = df['CommandLine'].str.len()
    
    #Only take the create action instead of open and terminate 
    df = df.drop(df[df['Action'] == 'OPEN'].index)
    df = df.drop(df[df['Action'] == 'TERMINATE'].index)
    
    #Convert datetime to time data
    df['timestamp_str'] = pd.to_datetime(df['Timestamp'])
    
    #Drop the old timestamp colum
    df.drop(columns=['Timestamp'], inplace=True)
    
    #Erstatt tomme felt med None
    df.fillna('None', inplace=True)
    
    #change ending of filename
    

    #rename ending of file
    fancyending = filename.replace("_process.csv", "_process_nodes.csv")
    
    #create the csv file with all attributes
    df.to_csv(fancyending,index=False)
    
    #rename ending of file
    fancyending = filename.replace("_process.csv", "_process_relations.csv")
    
    relations=df[['ParentProcessGuid', 'ProcessGuid']].copy() 

    relations.to_csv(fancyending,index=False)
    
    return fancyending


In [3]:
#try importing to add extra data
##modified 
##Drop inbound data
##Drop where destination is 127.0.0.1

import pandas as pd

def preprocess_flow_csv(filename):
    #
    df = pd.read_csv(filename,header=None)
    
    #create the correct headers
    df.columns = ["Action", "ProcessGuid", "Hostname", "unknown", "Object", "unkownProcessGuidish", "Pid", "Ppid", "Principal", "Acuity_level", "DestinationIp", "DestinationPort", "Initiated", "Image_path","Protocol", "size", "SourceIp", "SourcePort","Tid", "Timestamp"]
    
    
    #Only take the Message session
    df = df.drop(df[df['Action'] == 'MESSAGE'].index)
    df = df.drop(df[df['Action'] == 'TERMINATE'].index)
    df = df.drop(df[df['Action'] == 'OPEN'].index)
    #Only take outbound
    df = df.drop(df[df['Initiated'] == 'inbound'].index)
    #drop where destination is 127.0.0.1
    df = df.drop(df[df['DestinationIp'] == '127.0.0.1'].index)
    #df = df.drop(df[df['Action'] == 'OPEN'].index)
    #df = df.drop(df[df['Action'] == 'TERMINATE'].index)
    #df = df.drop(df[df['Action'] == 'START'].index)
    
    #Convert datetime to time data
    df['timestamp_str'] = pd.to_datetime(df['Timestamp'])
    
    #Drop the old timestamp colum
    df.drop(columns=['Timestamp'], inplace=True)
    
    #Erstatt tomme felt med None
    df.fillna('None', inplace=True)
    
    #change ending of filename
    #create the csv file with all attributes

    #rename ending of file
    fancyending = filename.replace("_flow.csv", "_network_node.csv")
    
    #create the csv file with all attributes
    df.to_csv(fancyending,index=False)
    
    #Extract only the unique processguids from the nodefile
    df_relations=df['ProcessGuid'].unique()
    
    #convert to a panda dataframe again
    relations = pd.DataFrame(df_relations)
    
    #Recreate the header name
    relations.columns = [ "ProcessGuid"]
        
    #Prepare the filename
    fancyending = filename.replace("_flow.csv", "_network_relations.csv")
     
    #create the csv file
    relations.to_csv(fancyending,index=False)
    
    
    return fancyending





In [4]:
#try importing to add extra data
import pandas as pd

def preprocess_file_csv(filename):
    #
    df = pd.read_csv(filename,header=None)
    
    #create the correct headers
    df.columns = ["Action", "ProcessGuid", "Hostname", "unknown", "Object", "Possible_ProcessGuid", "Pid", "Ppid", "Principal", "Acuity_level", "file_path", "Image_path", "info_class", "Sid", "User", "Tid", "Timestamp"]
    
    
    #Only take the create action instead of open and terminate 
    df = df.drop(df[df['Action'] == 'READ'].index)
    df = df.drop(df[df['Action'] == 'WRITE'].index)
    df = df.drop(df[df['Action'] == 'MODIFY'].index)
    df = df.drop(df[df['Action'] == 'DELETE'].index)
    df = df.drop(df[df['Action'] == 'RENAME'].index)
    
    #Convert datetime to time data
    df['timestamp_str'] = pd.to_datetime(df['Timestamp'])
    
    #Drop the old timestamp colum
    df.drop(columns=['Timestamp'], inplace=True)
    
    #Erstatt tomme felt med None
    df.fillna('None', inplace=True)
    
    #change ending of filename
    #create the csv file with all attributes

    #rename ending of file
    fancyending = filename.replace("_file.csv", "_file_nodes.csv")
    
    #create the csv file with all attributes
    df.to_csv(fancyending,index=False)
    
    #Extract only the unique processguids from the nodefile
    df_relations=df['ProcessGuid'].unique()
    
    #convert to a panda dataframe again
    relations = pd.DataFrame(df_relations)
    
    #Recreate the header name
    relations.columns = [ "ProcessGuid"]
        
    #Prepare the filename
    fancyending = filename.replace("_file.csv", "_file_relations.csv")
     
    #create the csv file
    relations.to_csv(fancyending,index=False)
    
    
    return fancyending



In [ ]:

import pandas as pd

def preprocess_shell_csv(filename):
    #
    df = pd.read_csv(filename,header=None)
    
    #create the correct headers
    df.columns = ["Action", "ProcessGuid", "Hostname", "unknown", "Object", "Unkown_ProcessGuid", "Pid", "Ppid", "Principal", "Acuity_level", "CommandLine", "Image_path", "amsi_log", "Sid", "User", "Tid", "Timestamp"]
    
    #Convert datetime to time data
    df['timestamp_str'] = pd.to_datetime(df['Timestamp'])
    
    #Drop the old timestamp colum
    df.drop(columns=['Timestamp'], inplace=True)
    
    #Drop the amsi log, no time to clean it up
    df.drop(columns=['amsi_log'], inplace=True)

    #extract the commandline into a temp panda dataframe
    dummy=df['CommandLine'].str.split('\n', expand=True)
    
    #rename the columns of the temp dataframe 
    dummy.columns = [ "Severity_level", "Host_type", "Console_Version", "Host_ID", "CommandLine", "Engine_Version", "Runspace_ID","Pipeline_ID", "Command_Name","Command_Type","Script_Name", "Command_Path", "Sequence_Number", "User", "Connected_User", "Shell_type"]
    #clean up the different rows removing what is no column header info
    dummy['Severity_level'] = dummy['Severity_level'].str.replace('Severity = ', '')
    dummy['Host_type'] = dummy['Host_type'].str.replace('Host Name = ', '')
    dummy['Console_Version'] = dummy['Console_Version'].str.replace('Host Version = ', '')
    dummy['Host_ID'] = dummy['Host_ID'].str.replace('Host ID = ', '')
    dummy['Command_Name'] = dummy['Command_Name'].str.replace('Command Name = ', '')
    dummy['Command_Type'] = dummy['Command_Type'].str.replace('Command Type = ', '')
    dummy['Script_Name'] = dummy['Script_Name'].str.replace('Script Name = ', '')
    dummy['CommandLine'] = dummy['CommandLine'].str.replace('Host Application = ', '')
    dummy['CommandLine'] = dummy['CommandLine'].str.replace('Host Application= ', '')
    dummy['Engine_Version'] = dummy['Engine_Version'].str.replace('Engine Version = ', '')
    dummy['Runspace_ID'] = dummy['Runspace_ID'].str.replace('Runspace ID = ', '')
    dummy['Command_Path'] = dummy['Command_Path'].str.replace('Command Path = ', '')
    dummy['Sequence_Number'] = dummy['Sequence_Number'].str.replace('Sequence Number = ', '')
    dummy['Pipeline_ID'] = dummy['Pipeline_ID'].str.replace('Pipeline ID = ', '')
    dummy['Connected_User'] = dummy['Connected_User'].str.replace('Connected User = ', '')
    dummy['User'] = dummy['User'].str.replace('User = ', '')
    dummy['Shell_type'] = dummy['Shell_type'].str.replace('Shell ID = ', '')
    
    #drop the old commandline from the original dataframe 
    df.drop(columns=['CommandLine'], inplace=True)
    #rename the dummy user to shell_user
    dummy.rename(columns={'User': 'Shell_User'}, inplace=True)
    #join them since they have the same index
    joined_df = df.join(dummy)
       
    #Erstatt tomme felt med None
    joined_df.fillna('None', inplace=True)
    
    #change ending of filename
    
    #rename ending of file
    fancyending = filename.replace("_shell.csv", "_shell_nodes.csv")
    
    #create the csv file with all attributes
    joined_df.to_csv(fancyending,index=False)
    
     #Extract only the unique processguids from the nodefile
    df_relations=joined_df['ProcessGuid'].unique()
    
    #convert to a panda dataframe again
    relations = pd.DataFrame(df_relations)
    
    #Recreate the header name
    relations.columns = ["ProcessGuid"]
        
    #Prepare the filename
    fancyending = filename.replace("_shell.csv", "_shell_relations.csv")
     
    #create the csv file
    relations.to_csv(fancyending,index=False)

    
    return fancyending

In [5]:
### Start with getting the list of file names of the different types of files
###################################################

In [6]:
#Get the current folder

In [7]:
%%time
#Get the current folder so that we only use the file in this folder
import os

current_folder = os.getcwd()
print(current_folder)

/home/jovyan/work/20-23
CPU times: user 88 µs, sys: 29 µs, total: 117 µs
Wall time: 139 µs


In [8]:
#try importing to add extra data
##Original 
#import pandas as pd

#def preprocess_flow_csv(filename):
    #
 #   df = pd.read_csv(filename,header=None)
    
 #   #create the correct headers
 #   df.columns = ["Action", "ParentProcessGuid", "Hostname", "unknown", "Object", "ProcessGuid", "Pid", "Ppid", "Principal", "Acuity_level", "DestinationIp", "DestinationPort", "Initiated", "Image_path","Protocol", "size", "SourceIp", "SourcePort","Tid", "Timestamp"]
    
    
    #Only take the Start session
 #   df = df.drop(df[df['Action'] == 'MESSAGE'].index)
 #   df = df.drop(df[df['Action'] == 'TERMINATE'].index)
 #   df = df.drop(df[df['Action'] == 'OPEN'].index)
    
    #Convert datetime to time data
 #   df['timestamp_str'] = pd.to_datetime(df['Timestamp'])
    
    #Drop the old timestamp colum
 #   df.drop(columns=['Timestamp'], inplace=True)
    
    #Erstatt tomme felt med None
 #   df.fillna('None', inplace=True)
    
    #change ending of filename
    #create the csv file with all attributes

    #rename ending of file
 #   fancyending = filename.replace("_flow.csv", "_network_node.csv")
    
    #create the csv file with all attributes
 #   df.to_csv(fancyending,index=False)
    
    
 #   return fancyending




In [9]:
%%time
import os
#Gets all process files and puts them in a variable
def get_process_files(directory):
    process_files = []
    for filename in os.listdir(directory):
        if filename.endswith('_process.csv'):
            process_files.append(filename)
    return process_files

process_files_result=get_process_files(current_folder)
print(process_files_result)

['AIA-501-525.ecar-last_process.csv']
CPU times: user 619 µs, sys: 203 µs, total: 822 µs
Wall time: 2.54 ms


In [10]:
%%time
import os
#Gets all flow node files and puts them in a variable
def get_network_flow_files(directory):
    network_flow_files = []
    for filename in os.listdir(directory):
        if filename.endswith('_flow.csv'):
             network_flow_files.append(filename)
    return network_flow_files

network_flow_result=get_network_flow_files(current_folder)
print(network_flow_result)

['AIA-501-525.ecar-last_flow.csv']
CPU times: user 0 ns, sys: 865 µs, total: 865 µs
Wall time: 524 µs


In [11]:
import os
#Gets all file_info files and puts them in a variable
def get_file_info_files(directory):
    file_info_files = []
    for filename in os.listdir(directory):
        if filename.endswith('_file.csv'):
            file_info_files.append(filename)
    return file_info_files

file_info_result=get_file_info_files(current_folder)
print(file_info_result)

['AIA-501-525.ecar-last_file.csv']


In [12]:
##Extract the data needed for the graph database
###################################################

In [13]:
%%time
#prepare the process_node files from flow files

for filename in process_files_result:
    print(filename)
    preprocess_process_csv(filename)
    
    

AIA-501-525.ecar-last_process.csv
CPU times: user 4.13 s, sys: 679 ms, total: 4.81 s
Wall time: 4.83 s


In [14]:
%%time
#prepare the network_node files from flow files

for filename in network_flow_result:
    print(filename)
    preprocess_flow_csv(filename)
    
    

AIA-501-525.ecar-last_flow.csv
CPU times: user 59.6 s, sys: 11.2 s, total: 1min 10s
Wall time: 1min 11s


In [15]:
%%time
#prepare the file_info files from file_info files

for filename in file_info_result:
    print(filename)
    preprocess_file_csv(filename)


AIA-501-525.ecar-last_file.csv
CPU times: user 16.1 s, sys: 1.56 s, total: 17.6 s
Wall time: 17.7 s


In [16]:
##Compress the flow/process/file files
###################################################

In [17]:
%%time
#compress the files since we don't need them in a while
import subprocess

for filename in network_flow_result:
    compressing_files="gzip "+filename
    subprocess.run(compressing_files, shell=True)
    

CPU times: user 1.82 ms, sys: 1.96 ms, total: 3.78 ms
Wall time: 1min 23s


In [18]:
%%time
#compress the files since we don't need them in a while
import subprocess

for filename in file_info_result:
    compressing_files="gzip "+filename
    subprocess.run(compressing_files, shell=True)

CPU times: user 1.83 ms, sys: 312 µs, total: 2.15 ms
Wall time: 18.3 s


In [19]:
%%time
#compress the files since we don't need them in a while
import subprocess

for filename in process_files_result:
    compressing_files="gzip "+filename
    subprocess.run(compressing_files, shell=True)

CPU times: user 1.56 ms, sys: 0 ns, total: 1.56 ms
Wall time: 5.48 s


In [20]:
###Move the files to the neo4j to be imported
###################################################

In [21]:
#Move the files to the import folders

!mv *_network_node.csv ../neo4j/import/20-23/
!mv *_network_relations.csv ../neo4j/import/20-23/
!mv *_file_nodes.csv ../neo4j/import/20-23/
!mv *_file_relations.csv ../neo4j/import/20-23/
!mv *_process_relations.csv ../neo4j/import/20-23/
!mv *_process_nodes.csv ../neo4j/import/20-23/